In [ ]:
# ------------------------------------------------------------------------------------------------------------------
# Amazon Rekognition is a service that makes it easy to add powerful visual analysis to your applications. 
# Rekognition Image lets you easily build powerful applications to search, verify, and organize millions of images. 
# Rekognition Video lets you extract motion-based context from stored or live stream videos and helps you analyze them.
# Rekognition detects objects, scenes, and faces; extracts text; recognizes celebrities; and identifies inappropriate 
# content in images. It also allows you to search and compare faces. Rekognition Image is based on the same proven, 
# highly scalable, deep learning technology developed by Amazonâs computer vision scientists to analyze billions of 
# images daily for Prime Photos.
# ------------------------------------------------------------------------------------------------------------------

In [ ]:
# -------------------------------------------------------------------------
# Import Python Libraries. Press run.
# -------------------------------------------------------------------------

from pyspark.context import SparkContext
import boto3
import requests
import uuid

In [ ]:
# -------------------------------------------------------------------------------------
# Create the Spark Context and a boto3 client for accessing rekognition API's, Press run.
# -------------------------------------------------------------------------------------

sc = SparkContext.getOrCreate()
client = boto3.client('rekognition', region_name='us-east-1')

BUCKET='aws-analytics-course'

In [ ]:
# --------------------------------------------------------------------------------------
# Lets perform a simple Rekognition. I have placed an image of a dog on S3. The dog
# is a Labrador Retriever. Let's see how Rekognition describes this image. Press run.
# --------------------------------------------------------------------------------------

photo='temp/rekognition/images/dog.jpg'

response = client.detect_labels(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}}, MaxLabels=10)

for label in response['Labels']:
    print ("Label: " + label['Name'])
    print ("Confidence: " + str(label['Confidence']))

In [ ]:
# --------------------------------------------------------------------------------------
# Note the labels attached to the image. A label is an object, scene, or concept found in 
# an image based on its contents.
# Also note the confidence for each lable. A confidence score is a number between 0 and 100 
# that indicates the probability that a given prediction is correct.
# Highlight next cell and Press run.
# --------------------------------------------------------------------------------------

In [ ]:
# --------------------------------------------------------------------------------------
# Lets perform another Rekognition. This time the image is of police officer standing in
# outside the subway station under a street sign.
# Highlight next cell and Press run.
# --------------------------------------------------------------------------------------

In [ ]:
photo='temp/rekognition/images/police-sign.jpg'

response = client.detect_labels(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}}, MaxLabels=10)

for label in response['Labels']:
    print ("Label: " + label['Name'])
    print ("Confidence: " + str(label['Confidence']))

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# How cool it would be if the street sign could be read as text. Imagine the possibilities that
# could open up. Text in Image is a capability of Amazon Rekognition that allows you to detect and recognize 
# text within an image, such as street names, captions, product names, and vehicular license plates.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
photo='temp/rekognition/images/police-sign.jpg'

response = client.detect_text(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}})

textDetections=response['TextDetections']
print ('Detected text')
for text in textDetections:
    print ('Detected text:' + text['DetectedText'])
    print ('Confidence: ' + "{:.2f}".format(text['Confidence']) + "%")
    print ('Id: {}'.format(text['Id']))
    if 'ParentId' in text:
        print ('Parent Id: {}'.format(text['ParentId']))
    print ('Type:' + text['Type'])
    print

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Notice the Detected Text fields above. Rekognition correctly detected the text from the street sign as
# 90 Saint.Eimhurst Av.. It also recognized another piece of text from a yellow parking lot sign.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Amazon Rekognitionâs Celebrity Recognition is a deep learning based easy-to-use API for detection and 
# recognition of individuals who are famous, noteworthy, or prominent in their field. Let's see how Celebrity 
# Recognition works.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
response = requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Bill_Gates_2017_%28cropped%29.jpg/220px-Bill_Gates_2017_%28cropped%29.jpg')
response_content = response.content

rekognition_response = client.recognize_celebrities(Image={'Bytes': response_content})

print(rekognition_response)

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Notice the Name tag in the output above. The image was correctly identified as famous physicist Stephen Hawking.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
url='https://upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Shah_Rukh_Khan_graces_the_launch_of_the_new_Santro.jpg/220px-Shah_Rukh_Khan_graces_the_launch_of_the_new_Santro.jpg'
    
response = requests.get(url)
response_content = response.content

rekognition_response = client.recognize_celebrities(Image={'Bytes': response_content})

print(rekognition_response)

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# This time the image was correctly identified as famous actor Shahrukh Khan.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Can Amazon Rekognition compare faces? It can. Face Comparison is the process of comparing one face to one 
# or more faces to measure similarity. I have uploaded 2 images of Tiger Woods for comparison. Let's see 
# how these faces compare.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
KEY_SOURCE = "temp/rekognition/images/tiger1.jpg"
KEY_TARGET = "temp/rekognition/images/tiger2.jpg"

def compare_faces(bucket, key, bucket_target, key_target, threshold=80, region="us-east-1"):
    rekognition = boto3.client("rekognition", region)
    response = client.compare_faces(
    SourceImage={
          "S3Object": {
          "Bucket": bucket,
          "Name": key,
          }
        },
        TargetImage={
           "S3Object": {
           "Bucket": bucket_target,
           "Name": key_target,
           }
        },
        SimilarityThreshold=threshold,
        )
    return response['SourceImageFace'], response['FaceMatches']


source_face, matches = compare_faces(BUCKET, KEY_SOURCE, BUCKET, KEY_TARGET)

# the main source face
print("Source Face ({Confidence}%)".format(**source_face))
#print "matches: "+str(matches)

# one match for each target face
for match in matches:
    print("Target Face ({Confidence}%)".format(**match['Face']))
    print("  Similarity : {}%".format(match['Similarity']))


In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Amazon Rekognition compared the faces and came to the conclusion that the two images are a perfect match 
# of Tiger Woods.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# This time lets compare Tiger Woods to a Tiger Woods look alike.
# I have uploaded the image of Tiger Woods and his look alike for comparison. Let's see 
# how these faces compare.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
KEY_SOURCE = "temp/rekognition/images/tiger1.jpg"
KEY_TARGET = "temp/rekognition/images/tiger_not.jpg"

def compare_faces(bucket, key, bucket_target, key_target, threshold=80, region="eu-west-1"):
    rekognition = boto3.client("rekognition", region)
    response = client.compare_faces(
    SourceImage={
          "S3Object": {
          "Bucket": bucket,
          "Name": key,
          }
        },
        TargetImage={
           "S3Object": {
           "Bucket": bucket_target,
           "Name": key_target,
           }
        },
        SimilarityThreshold=threshold,
        )
    return response['SourceImageFace'], response['FaceMatches']


source_face, matches = compare_faces(BUCKET, KEY_SOURCE, BUCKET, KEY_TARGET)

# the main source face
#print "Source Face ({Confidence}%)".format(**source_face)
if not matches:
    print("Not a Match")


# one match for each target face
for match in matches:
    print("Target Face ({Confidence}%)".format(**match['Face']))
    print("  Similarity : {}%".format(match['Similarity']))


In [ ]:
# ----------------------------------------------------------------------------------------------------------
# It's not a match!!
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
COLLECTION = "rek-collection-1"
client.create_collection(CollectionId=COLLECTION)
client.list_collections()

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Amazon Rekognition can store information about detected faces in server-side containers known as collections. 
# You can use the facial information that's stored in a collection to search for known faces in images, stored 
# videos, and streaming videos. Let' store the image of Tiger Woods in a new collection.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
KEY = "temp/rekognition/images/tiger1.jpg"
IMAGE_ID = uuid.uuid4().hex  

# Note: you have to create the collection first!
#client.create_collection(CollectionId=COLLECTION)

def index_faces(bucket, key, collection_id, image_id=None, attributes=(), region="us-east-1"):

    response = client.index_faces(
                                 Image={
                                 "S3Object": {
                                 "Bucket": bucket,
                                 "Name": key,
                                 }
                                 },
                                 CollectionId=collection_id,
                                 ExternalImageId=image_id,
                                     DetectionAttributes=attributes,
                                 )
    return response['FaceRecords']


for record in index_faces(BUCKET, KEY, COLLECTION, IMAGE_ID):
    face = record['Face']
    # details = record['FaceDetail']
    print("Face ({}%)".format(face['Confidence']))
    print("  FaceId: {}".format(face['FaceId']))
    print("  ImageId: {}".format(face['ImageId']))


In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Now let's search for Tiger Woods in our collections database using a completely new picture.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
KEY = "temp/rekognition/images/search_tiger.jpg"

def search_faces_by_image(bucket, key, collection_id, threshold=80, region="eu-west-1"):
    response = client.search_faces_by_image(
                         Image={
                         "S3Object": {
                         "Bucket": bucket,
                         "Name": key,
                         }
                         },
                         CollectionId=collection_id,
                         FaceMatchThreshold=threshold,
                         )
    return response['FaceMatches']

for record in search_faces_by_image(BUCKET, KEY, COLLECTION):
        face = record['Face']
        print("Matched Face ({}%)".format(record['Similarity']))

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Looks like the picture matched. Comparing faces within collections goes a long way in Master Data 
# Management  initiatives.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Now let's search for Will Smith in our collections database.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
KEY = "temp/rekognition/images/will.jpg"

def search_faces_by_image(bucket, key, collection_id, threshold=80, region="eu-west-1"):
    response = client.search_faces_by_image(
                 Image={
                      "S3Object": {
                      "Bucket": bucket,
                      "Name": key,
                 }
                 },
                 CollectionId=collection_id,
                 FaceMatchThreshold=threshold,
                 )
    return response['FaceMatches']

for record in search_faces_by_image(BUCKET, KEY, COLLECTION):
        face = record['Face']
        print("Matched Face ({}%)".format(record['Similarity']))
        print("  FaceId : {}".format(face['FaceId']))
        print("  ImageId : {}".format(face['ExternalImageId']))
else:
    print('Image not in our database')

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Will Smith does not exist in our collections database.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
client.delete_collection(CollectionId=COLLECTION)
client.list_collections()

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# How can object detection within an image be useful? Imagine a security camera captured a man with a gun in a
# busy mall. Image labels can be read by Data Science/Monitoring Algorithms and proper authorities could be 
# alerted before a mishap happens. 
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
photo='temp/rekognition/images/pistol.jpg'

response = client.detect_labels(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}}, MaxLabels=10)

for label in response['Labels']:
    print ("Label: " + label['Name'])
    print ("Confidence: " + str(label['Confidence']))

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# The possibilities are endless. Everyday many large scale organizations are finding new ways to use this 
# technology. I hope you found this information both informative and useful. Amazon 
# Rekognition is covered in more detail in the AWS Data Analytics Speciality training. 
# ----------------------------------------------------------------------------------------------------------